In [1]:
import pandas as pd
from collections import Counter
df = pd.read_csv("customer_feedback_clean.csv")
neg = df[df['label']=='Negative']
words = " ".join(neg['processed'].astype(str)).split()
cnt = Counter(words)
top10 = cnt.most_common(10)
print("Top issues:", top10)


Top issues: [('product', 10), ('want', 5), ('refund', 5), ('device', 5), ('stopped', 5), ('working', 5), ('week', 5), ('user', 5), ('delivery', 5), ('delayed', 5)]


In [ ]:
import pandas as pd
from prophet import Prophet

df = pd.read_csv("customer_feedback_clean.csv", parse_dates=['timestamp'] if 'timestamp' in pd.read_csv('customer_feedback_clean.csv').columns else None)
if 'timestamp' not in df.columns:
    import numpy as np
    rng = pd.date_range(end=pd.Timestamp.today(), periods=len(df))
    df['timestamp'] = rng
score_map = {"Positive":1,"Neutral":0.5,"Negative":0}
df['score'] = df['label'].map(score_map)
daily = df.groupby(df['timestamp'].dt.date)['score'].mean().reset_index()
daily.columns = ['ds','y']
m = Prophet()
m.fit(daily)
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)
pred_next_month = forecast.tail(30)['yhat'].mean()
print("Predicted average satisfaction score next 30 days:", pred_next_month)
